# OPPGAVE 1 KLASSIFISERING

#### Bruk datasettet Universalbank.csv og bygg klassifiseringsmodeller som gir en binær prediksjon om en kunde vil akseptere et lån fra Universal Bank eller ikke.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split


# 1 Filopplastning

### 1.1 Lesing
Her har jeg laget en egen variabel for file path, slik at hvis jeg bruker den flere steder og den senere skulle endre til et annet datasett, så blir endringen lettere. 

### 1.2 Visning
Jeg printer også toppradene til dataene/tabellen for å få et overblikk over hvilke ulike typer data vi har å jobbe med. Det gjør det blandt annet lettere for meg å se om det er noen unødvendige kolonner med data som kan fjernes, ser at det ikke mangler essensielle kolonner (som "personal loan" i denne oppgaven).

In [4]:
# 1.1 Lesing
file_path = 'Data/UniversalBank.csv'
df = pd.read_csv(file_path)

# 1.2 Vising
print("\n Tabell første rader")
print(df.head())


 Tabell første rader
   ID  Age  Experience  Income  ZIP Code  Family  CCAvg  Education  Mortgage  \
0   1   25           1      49     91107       4    1.6          1         0   
1   2   45          19      34     90089       3    1.5          1         0   
2   3   39          15      11     94720       1    1.0          1         0   
3   4   35           9     100     94112       1    2.7          2         0   
4   5   35           8      45     91330       4    1.0          2         0   

   Personal Loan  Securities Account  CD Account  Online  CreditCard  
0              0                   1           0       0           0  
1              0                   1           0       0           0  
2              0                   0           0       0           0  
3              0                   0           0       0           0  
4              0                   0           0       0           1  


# 2 Kvalitetssjekk

For at modellene skal bli riktige/best mulig, må man sjekke at man har riktige og gode data. 

### 2.1 Fordeling
Her sjekker jeg om det finnes både ja og nei til lån og hvordan andelene er fordelt. I dette datasettet ser vi at det er 10% som er Ja/1 og retserende er nei/0. Denne skjeivfordelingen kan skape problemer ved trening og testing om man ikke tar hennsyn til det. Så denne delen av koden er for å oppdage dette slik at vi ikke ignorerer det. 

### 2.2 Kvalitetssjekk målvariabel
Videre vil jeg sjekke at vi har det vi trenger fra målvariabelen vår. Vi skal finne ut av om en person får lån eller ikke, som vil si at målvariabelen blir "Personal Loan". Her vil jeg sjekke om alle dataene er oppgitt likt. Om noen data skulle mangle eller være ugjyldige, eller si "yes" isteden for 1 eller liknende, må disse dataene fjernes eller rettes på før vi går videre på treningen. Vi kan ikke bare fjerne hele kolonnen og må ta ekstra hennsyn til denne, da hele treningen avhenger av denne kolonnen/dataene. Jeg sjekket målvariablene for seg da de er viktige, slik at det skal være lettere å fikse opp i eventuelle feil, samt gjøre det lettere å huske på.

### 2.3 Kvalitetssjekk variabler
De andre ataene variablene er ikke nødvendighvis like avgjørende for at modellen skal fungere godt og sees derfor på separat fra målvariabelen. Om det skulle være feil eller mangler ved disse kan man finne løsninger avhengig av feilen. Men jeg velger å sjekke dette for å se hvor det eventuellt ligger feil for å kunne avgjøre om dette er noe som kan påvirke dataene og må endres på. 

### 2.4 Datatyper
Jeg sjekker datatypene i hver kolonne for å få overordnet kontroll og få inblikk i hvordan behandle dataene videre. Eks kan det være greit å vite datatypen til hvert felt, men også være klar over dersom ett enkelt felt inneholder flere datatyper. Dersom et felt eks skriver "ja" og "nei" i strenger, vil jeg kanskje endre dette til binær. Om det eks er en type variabel som oppgir dataene som string "ja" i noen data og som "1" i binær andre, så må jeg også ta hennsyn til dette og gjøre de like. Altså vil jeg skape overordnet kontroll over hva slags data jeg sitter med, slik jeg kan tilpasse dataene etter hvordan jeg ønsker/trenger dem til læringen. 

### 2.5 Duplikater
Duplikater kan skape problemer og bør fjernes. I denne koden ser jeg ikke umiddelbare ulemper med duplikatene, men jeg ser heller ingen fordel, og hadde derfor fjernet dem om det hadde fantes. Jeg mistenker at det poensiellt kunne skapt bias i løsningen med trær f.eks. 

In [5]:

# 2.1 Fordeling
print("\n Antall per klasse av målvariabel")
print(df['Personal Loan'].value_counts())

print("\n Andel per klasse i poosent (%) av målvariabel")
print(df['Personal Loan'].value_counts(normalize=True))

# 2.2 Kvalitetssjekk målvariabel
print("\n Unike verdier av målvariabel")
print(df['Personal Loan'].unique())

print("\n Antall manglende verdier av målvariabel")
print(df['Personal Loan'].isna().sum()) 

# 2.3 Kvalitetssjekk variabler
print("\nManglende verdier per kolonne:")
print(df.isna().sum())

# 2.4 Datatyper
print("\nDatatyper per kolonne:")
print(df.dtypes)

# 2.5 Duplikater
print("\n Antall duplikatrader:", df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True)


 Antall per klasse av målvariabel
Personal Loan
0    4520
1     480
Name: count, dtype: int64

 Andel per klasse i poosent (%) av målvariabel
Personal Loan
0    0.904
1    0.096
Name: proportion, dtype: float64

 Unike verdier av målvariabel
[0 1]

 Antall manglende verdier av målvariabel
0

Manglende verdier per kolonne:
ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

Datatyper per kolonne:
ID                      int64
Age                     int64
Experience              int64
Income                  int64
ZIP Code                int64
Family                  int64
CCAvg                 float64
Education               int64
Mortgage                int64
Personal Loan           int64
Se

# 3. Rensing og inndeling

Ut ifra funnene i kvalitetssjekken så renser jeg dataene. I kvalitetssjekken fant jeg ingen store feil i dataene. 

### 3.1 Rensing
Det er i midlertid visse ting som bør renses av logiske grunner. 
- ID vil ikke ha noe med om en person får lån eller ikke, og skal derfor fjernes/droppes. 
- Hvor man bor burde logisk sett heller ikke ha noe med om man får lån eller ikke, derfor fjerner/dropper jeg den også. Det kan finnes scenarier hvor dette har noe å si, men da må man kode mer rundt for å ta i bruk denne variabelen.

### 3.2 Inndeling
Det deles videre inn i X (features) og y (målvariabel). Målvariabelen fjernes fra features fordi vi vil unngå at man "ser" fasiten under trening, men kun bruker den for å justere modellen etter å ha truffet/bommet. 

### 3.3 Train/test fordeling
Vi bruker stratify for å bevare samme prosentfordeling mellom testdataene og treningsdataene, slik at det ikke skal komme feil som følge av skeivfordeling. Vi printer deretter ut hvor godt fordelt det er for å sjekke at alt gikk riktig. Dataene deles i 80/20 da vi har nok data å trene på til å ikke måtte gå opp til 90/10, men det kan hende at også 70/30 kan være en fin fordeling da vi har god mengde med data. 


In [21]:
from sklearn.model_selection import train_test_split

# 3.1 Rensing. 
cols_to_drop = ['ID', 'ZIP Code']
df_clean = df.drop(columns=cols_to_drop).copy()

# 3.2 Inndeling
X = df_clean.drop(columns=['Personal Loan'])
y = df_clean['Personal Loan']

# 3.3 Train/test fordeling
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

print("\n Andel lån akseptert :")
print("Total :", y.mean().round(3))
print("Train :", y_train.mean().round(3))
print("Test  :", y_test.mean().round(3))




 Andel lån akseptert :
Total : 0.096
Train : 0.096
Test  : 0.096


# 4. Valg av modell

Da vi vet utfallet på dataene vi jobber med, jobber vi med suprivised learning, hvor vi bruker utfallet som fasit for å sjekke om prediksjonene er riktig. Altså har vi en målestokk og faktisk fasit på hvor godt modellen treffer. 

Noen modeller som ble vurdert var:

1. Logistisk regresjon
Denne modellen er lett og vanlig på datasett hvor målvariabelen er ja/nei. Man kan si for hver kolonne, så øker eller minker sannsynligheten for at utfallet går mot ja eller nei, altså over eller under en gitt grense/strek. Denne er best når sammenhengen mellom variabler og utfall er linjær. Jeg vil tro at om en person har god inntekt, så vil beslutningen naturlighvis lene mer mot et ja. På samme måte som hvis personen allerede har mye lån, så leder naturlig beslutningen mer mot nei. Altså øker/minker sannsynligheten for et gitt utfall, ved hvert utsagn/felt. Denne sliter mer når det er ting som ikke alltid har en like åpenbar vekt mellom ja/nei. 

2. Bessutningstre
Et beslutningstre går gjennom variablene en og en og lager ja/nei-splitt. Problemet er at det lett kan overfitte. Vi har mange tallfelt og mange mulige terskler, så treet kan ende opp med veldig spesielle regler som bare passer en liten gjeng i treningsdata og ikke nødvendighvis passer nye kudner så godt.  Jo flere mulige splitt, desto flere grener, og da blir modellen spiss og mindre robust. I tillegg er "ja til lån" en liten gruppe. Da kan treet fort lage egne smale regler for å fange disse få tilfellene, i stedet for å lære det generelle mønsteret. Den er bedre enn lineær ved at den kan finne sammenheng i kombinasjoner. Eks 1 betyr ja, 2 betyr nei i lineær, men kanskje kombinasjonen av begge vekter mot nei, og dette kan denne plukke opp på. 


3. Randome Forest
Denne modellen er også mer vanlig i supervised modeller (trenger X og y). Random Forest består av mange små beslutningstrær som hver får litt tilfeldige data/variabler. Hvert tre stemmer, og flertallet bestemmer. Fordelen vs. ett tre er at veldig spissede (overtilpassede) regler ikke får dominere, fordi skogen jevner ut disse utslagene. Resultatet blir ofte mer stabil modell med mindre overfitting. 

4. KNN
KNN regner ut avstand mellom punkter og bruker verdiene i feltene til å beregne disse avstandene. Denne ryker da, fordi vi har felter med 1 og 0, og vi har felter som er høye nummer som lønn. Avstanden vil derfor bli beregnet veldig i fordel til visse felter, og kan ta for lite hennsyn til de andre feltene som kanskje er avgjørende. Man kunne potensiellt også endret på formatet på dataene om man ville bruke denne ved å kategorisere lønn i grupper f.eks, men dette så jeg som mer komplisert enn det som var nødvendig for opppgaven. 

5. NN
Nevrale nett er også mulig, men blir for komplisert i denne oppgaven. En av fordelene med NN er at man ikke trenger å vite svaret for å kunne finne mønster, men når man har mønster så er det lettere å bruke en metode som utnytter dette. 

En siste mulig løsning er å kombinere flere modeller om man skulle øsnke det. 

Valget ble derfor å gå med en modell som bruker Logistisk Regresjon, og en som bruker Beslutningstre.


# 5. Modell 1, Logistisk Regresjon

### 5.1 Trening
Siden dataene våre har stor ubalanse (få "ja") brukers det class_weight = "balanced" fordi modellen ellers "jukser" ved å nesten alltid gjette flertallet (nei). Balanced gjør straffen for å bomme på den lille gruppen (ja) større, og straffen for å bomme på den store gruppen (nei) mindre. Dette gjør at modellen må jobbe mer for å finne den lille gruppen, i stedet for å ignorere dem for å få høy (men meningsløs) treffsikkerhet.

### 5.2 Prediksjoner
Denne delen gjør to ting. Først så gir den en avgjørelse ja (1) eller nei (0). Deretter sjekkes det hvor sannsynlig det er at dette utfallet er riktig. Altså kan utfallet være ja, men kun med 60% sannsynlighet. Den ene brukes kun for å se svaret, og den andre for å evaluere treffsikkerheten. 

### 5.3 Evaluering
I evalueringen ser vi på :
1. Nøyaktigheten: Som forteller oss om antall riktige vi har fått totalt
2. Presisjon: som sjekker på hvor mange av "ja" dataene er faktisk riktige. True positive (TP)
3. Recall: Hvor mange av ja kundene den klarer å finne
4. F1 : Balanserer presisjon og recall (Da disse påvirker hverandre)
5. ROC-AUC : Sier noe om hvor godt modellen kan skille mellom ja/nei, uansett terskel. 


### 5.4 Forvirringsmatrise/Confusion matrix
Tilslutt valgte jeg å ha med en matrise som viser noe om nøyaktigheten. Det er greit å vite hvor mange "false positives" (FP) og "False negatives" (FN) man har, for å kunne se hva modellen gjør.

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix
)

# 5.1 Trening
logit = LogisticRegression(class_weight='balanced', max_iter=1000 )
logit.fit(X_train, y_train)

# 5.2 Prediksjoner

y_pred = logit.predict(X_test)
y_prob = logit.predict_proba(X_test)[:, 1]

# 5.3 Evaluering todo Evaluer (ikke bare accuracy pga. ubalanse), hva vil dette si?
print("\n Evaluering/Resultater")
print("Accuracy :", round(accuracy_score(y_test, y_pred), 3))
print("Precision:", round(precision_score(y_test, y_pred), 3))
print("Recall   :", round(recall_score(y_test, y_pred), 3))
print("F1-score :", round(f1_score(y_test, y_pred), 3))
print("ROC-AUC  :", round(roc_auc_score(y_test, y_prob), 3))

# 5.4 Confusion matrix (TP/FP/FN/TN)
print("\nConfusion matrix (TN, FP; FN, TP):")
print(confusion_matrix(y_test, y_pred))


 Evaluering/Resultater
Accuracy : 0.896
Precision: 0.479
Recall   : 0.938
F1-score : 0.634
ROC-AUC  : 0.962

Confusion matrix (TN, FP; FN, TP):
[[806  98]
 [  6  90]]


C:\Users\melo1\OneDrive - NTNU\Desktop\Alle semester\5 semester\Big Data\DataScienceArbeidskrav\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# 6. Modell 2, Randome Tree

### 6.1 Trening
Koden lager mange små trær og trener dem på litt forskjellige biter av data. Hvert tre lærer sine egne enkle regler. Når alle trærne er trent, er modellen klar til å stemme på svar senere. Poenget i koden er altså: bygg mange trær, la dem lære hver for seg, og gjør dem klare til felles avstemning.

### 6.2 Prediksjoner
Denne delen gjør to ting. Først så gir den en avgjørelse ja (1) eller nei (0). Deretter sjekkes det hvor sannsynlig det er at dette utfallet er riktig. Altså kan utfallet være ja, men kun med 60% sannsynlighet. Den ene brukes kun for å se svaret, og den andre for å evaluere treffsikkerheten. 

### 6.3 Evaluering
I evalueringen ser vi på :
1. Nøyaktigheten: Som forteller oss om antall riktige vi har fått totalt
2. Presisjon: som sjekker på hvor mange av "ja" dataene er faktisk riktige. True positive (TP)
3. Recall: Hvor mange av ja kundene den klarer å finne
4. F1 : Balanserer presisjon og recall (Da disse påvirker hverandre)
5. ROC-AUC : Sier noe om hvor godt modellen kan skille mellom ja/nei, uansett terskel. 


### 6.4 Forvirringsmatrise/Confusion matrix
Tilslutt valgte jeg å ha med en matrise som viser noe om nøyaktigheten. Det er greit å vite hvor mange "false positives" (FP) og "False negatives" (FN) man har, for å kunne se hva modellen gjør.

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score,
                             recall_score, f1_score, roc_auc_score,
                             confusion_matrix)

# 6.1 Trening
rf = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced_subsample',
    random_state=42,
    n_jobs=-1 
)

rf.fit(X_train, y_train)

# 6.2 Prediksjon
y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:, 1]

# 6.3 Evaluering/Resultat
print("\nEvaluering – Random Forest")
print("Accuracy :", round(accuracy_score(y_test, y_pred), 3))
print("Precision:", round(precision_score(y_test, y_pred), 3))
print("Recall   :", round(recall_score(y_test, y_pred), 3))
print("F1-score :", round(f1_score(y_test, y_pred), 3))
print("ROC-AUC  :", round(roc_auc_score(y_test, y_prob), 3))

# 6.4 Matrise
print("\nConfusion matrix (TN, FP; FN, TP):")
print(confusion_matrix(y_test, y_pred))


NameError: name 'X_train' is not defined

# 7 DISKUSJON

### Evaluering - Logistisk reggresjon
Accuracy : 0.899
Precision: 0.486
Recall   : 0.927
F1-score : 0.638
ROC-AUC  : 0.965

Confusion matrix (TN, FP; FN, TP):
[[810  94]
 [  7  89]]
 
### Evaluering – Random Forest
Accuracy : 0.99
Precision: 0.957
Recall   : 0.938
F1-score : 0.947
ROC-AUC  : 0.998

Confusion matrix (TN, FP; FN, TP):
[[900   4]
 [  6  90]]
 
### Hvem gjør best?
Random Forest vinner på alle målene. Logistisk regresjon har også bra resultater, men har lav presisjon. Dette vil si at den nesten finner alle "ja" utfallene, men kommer ofte med FP og sier "ja" når svaret egt er nei. På FN så er resultatene like mellom modellene, men på den FP så gjorde Random forest det 95% bedre. 

### Hvorfor?
Jeg tenkte at hver faktor alene spiller en rolle på om en person får lån eller ikke, men da RF hadde bedre resultater, vil jeg annta at det er sammenhenger mellom faktorene som RF klarte å plukke opp på som LR ikke klarte. Jeg vil fortsatt si at RF gjorde det bedre nå enn det et vanlig tre hadde gjort, da den tar bedre hennsyn til denne ubalansen mellom ja og nei. og LR er fortsatt en relativt god modell med gode tall, men den er langt nær like god som RF. 

### Tidlige valg
Sjekken av dataene gikk fint, så det ble ikke gjort noen endirnger i forhold til dem. Om det hadde blitt gjort endirnger, så vil jeg tro at disse, sammen med endringene om å fjerne 2 ikke nødvendige felter, kun gjorde modellene bedre da de mer sannsynlig fjernet støy. Det jeg kommer på som eventuellt kunne hatt noe å si er ZIP, hvor det er mulig at det har noen sammenheng, men når jeg kjørte samme kode med ZIP inne i datasettet, forble resultatet uendret, som vil si at for denne oppgaven så hadde det ingenting å si. 

